In [1]:
import os
from pathlib import Path
import json

import numpy as np 
import pandas as pd
import math
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import cv2
import json
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.size"] = 10
plt.rcParams['figure.figsize'] = (15, 5)
import seaborn as sns
from PIL import Image

from sklearn.model_selection import train_test_split
import scipy
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score

# Tensorflow
#from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras

from collections import Counter
from collections import defaultdict

from keras.models import Model, Sequential
from keras.optimizers import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Input
from keras.utils import to_categorical
from keras.preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
from tensorflow.keras.optimizers import Adam

In [2]:
input_dir = "../input/severstal-steel-defect-detection"
train_dir = os.path.join(input_dir,"train_images")
test_dir = os.path.join(input_dir,"test_images")

In [3]:
train_df = pd.read_csv(os.path.join(input_dir, "train.csv"), encoding='utf-8')
train_df.head()

In [4]:
train_df['fname'], train_df['cls'] = train_df['ImageId'], train_df['ClassId']
train_df['cls'] = train_df['cls'].astype(int)
train_df = train_df.pivot(index='fname',columns='cls',values='EncodedPixels')
train_df['defects'] = train_df.count(axis=1)
train_df.head()

In [5]:
df_multilabel = pd.DataFrame()
df_multilabel['defect1'] = train_df[1].notnull().astype('int')
df_multilabel['defect2'] = train_df[2].notnull().astype('int')
df_multilabel['defect3'] = train_df[3].notnull().astype('int')
df_multilabel['defect4'] = train_df[4].notnull().astype('int')
df_multilabel.head()

In [6]:
df_multilabel['filenames'] = df_multilabel.index
df_multilabel.reset_index(drop=True, inplace=True)
df_multilabel.head()

In [7]:
def show_image(imgId):
    img = cv2.imread(os.path.join(train_dir, imgId))
    print('Image size', img.shape)
    plt.imshow(img)
    plt.show()
    
show_image(df_multilabel['filenames'][5])

# DATA GENERATION

In [8]:
train_ml, test_ml = train_test_split(df_multilabel, )
train_ml, test_ml = train_test_split(df_multilabel, test_size = 0.1, random_state=42)
train_ml, val_ml = train_test_split(train_ml, test_size = 0.2, random_state=42)
print(train_ml.shape, val_ml.shape, test_ml.shape)

In [9]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

columns=["defect1", "defect2", "defect3", "defect4"]
train_generator=datagen.flow_from_dataframe(train_ml,
                                            directory=train_dir,
                                            x_col='filenames',
                                            y_col=columns,
                                            batch_size=16,
                                            class_mode='raw',
                                            seed=42,
                                            shuffle=True,
                                            target_size=(128, 256))

valid_generator=datagen.flow_from_dataframe(dataframe=val_ml,
                                                directory=train_dir,
                                                x_col='filenames',
                                                y_col=columns,
                                                batch_size=16,
                                                seed=42,
                                                class_mode="raw",
                                                shuffle=True,
                                                target_size=(128, 256))

test_generator=test_datagen.flow_from_dataframe(dataframe=test_ml,
                                                directory=train_dir,
                                                x_col='filenames',
                                                batch_size=1,
                                                seed=42,
                                                class_mode=None,
                                                shuffle=False,
                                                target_size=(128, 256))

# CNN Model for Multilabel Classification with single output

In [42]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(128, 256, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

In [43]:
earlystopper = EarlyStopping(monitor='val_acc', patience=10, verbose=1)

# Save the best model during the traning
checkpointer = ModelCheckpoint('best_model1.h5',
                               monitor='val_acc',
                               verbose=1,
                               save_best_only=True,
                               save_weights_only=True)

In [44]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

history = model.fit_generator(generator=train_generator,
                            steps_per_epoch=STEP_SIZE_TRAIN,
                            validation_data=valid_generator,
                            validation_steps=STEP_SIZE_VALID,
                            epochs=30,
                            callbacks=[earlystopper, checkpointer])

In [45]:
test_generator.reset()
y_pred = model.predict_generator(test_generator, steps=STEP_SIZE_TEST)
pred_bool = (y_pred>0.5)
y_pred = pred_bool.astype(int)
y_pred

In [46]:
y_true = test_ml[["defect1", "defect2", "defect3", "defect4"]].values
y_true = y_true[:len(y_pred)]
y_true

In [47]:
class Evaluator:
    """
    Predict on test data (not submission data from test folder) and print reports, plot results etc.
    """
     
    def __init__(self, model, training, generator, y_true):
        self.training = training
        self.generator = generator
        
        # predict the data
        self.generator.reset()
        self.y_pred_raw = model.predict_generator(self.generator, steps=STEP_SIZE_TEST)
        self.pred_bool = (self.y_pred_raw>0.5)
        self.y_pred = self.pred_bool.astype(int)
        self.y_true=y_true[:len(self.y_pred)]  
        
    
    def plot_history(self):
        """
        Plot training history
        """
        ## Trained model analysis and evaluation
        f, ax = plt.subplots(1,2, figsize=(12,3))
        ax[0].plot(self.training.history['loss'], label="Loss")
        ax[0].plot(self.training.history['val_loss'], label="Validation loss")
        ax[0].set_title('Loss')
        ax[0].set_xlabel('Epoch')
        ax[0].set_ylabel('Loss')
        ax[0].legend()

        # Accuracy
        ax[1].plot(self.training.history['acc'], label="Accuracy")
        ax[1].plot(self.training.history['val_acc'], label="Validation accuracy")
        ax[1].set_title('Accuracy')
        ax[1].set_xlabel('Epoch')
        ax[1].set_ylabel('Accuracy')
        ax[1].legend()
        plt.tight_layout()
        plt.show()
        
    def Accuracy(self):
        temp = 0
        for i in range(self.y_true.shape[0]):
            temp += sum(np.logical_and(self.y_true[i], self.y_pred[i])) / sum(np.logical_or(self.y_true[i], self.y_pred[i]))
        return temp / self.y_true.shape[0]

    def Precision(self):
        temp = 0
        for i in range(self.y_true.shape[0]):
            if sum(self.y_true[i]) == 0:
                continue
            temp+= sum(np.logical_and(self.y_true[i], self.y_pred[i]))/ sum(self.y_true[i])
        return temp/ self.y_true.shape[0]

    def Recall(self):
        temp = 0
        for i in range(self.y_true.shape[0]):
            if sum(self.y_pred[i]) == 0:
                continue
            temp+= sum(np.logical_and(self.y_true[i], self.y_pred[i]))/ sum(self.y_pred[i])
        return temp/ self.y_true.shape[0]

    def F1Measure(self):
        temp = 0
        for i in range(self.y_true.shape[0]):
            if (sum(self.y_true[i]) == 0) and (sum(self.y_pred[i]) == 0):
                continue
        temp+= (2*sum(np.logical_and(self.y_true[i], self.y_pred[i])))/ (sum(self.y_true[i])+sum(self.y_pred[i]))
        return temp/ self.y_true.shape[0]

In [48]:
# Create evaluator instance
evaluator = Evaluator(model, history, test_generator, test_ml[["defect1", "defect2", "defect3", "defect4"]].values)

# Draw accuracy and loss charts
evaluator.plot_history()

print('Test Accuracy', evaluator.Accuracy())
print('Test Precision', evaluator.Precision())
print('Test Recall', evaluator.Recall())
print('Test F1score', evaluator.F1Measure())

# Multioutput Model

In [49]:
inp = Input(shape=(128, 256, 3))
x = Conv2D(32, (3, 3), padding = 'same')(inp)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(64, (3, 3), padding = 'same')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Flatten()(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
output1 = Dense(1, activation = 'sigmoid')(x)
output2 = Dense(1, activation = 'sigmoid')(x)
output3 = Dense(1, activation = 'sigmoid')(x)
output4 = Dense(1, activation = 'sigmoid')(x)
model1 = Model(inp, [output1, output2, output3, output4])
model1.compile(optimizers.rmsprop(lr = 0.0001, decay = 1e-6),
loss = ["binary_crossentropy", "binary_crossentropy", "binary_crossentropy", "binary_crossentropy"], metrics = ["accuracy"])

In [50]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(4)])

In [51]:
earlystopper = EarlyStopping(monitor='val_acc', patience=10, verbose=1)

# Save the best model during the traning
checkpointer = ModelCheckpoint('best_model2.h5',
                               monitor='val_acc',
                               verbose=1,
                               save_best_only=True,
                               save_weights_only=True)

In [53]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

history_multioutput = model1.fit_generator(generator=generator_wrapper(train_generator),
                                            steps_per_epoch=STEP_SIZE_TRAIN,
                                            validation_data=generator_wrapper(valid_generator),
                                            validation_steps=STEP_SIZE_VALID,
                                            epochs=10,
                                            verbose=1,
                                            callbacks=[earlystopper, checkpointer])

In [54]:
class Evaluate:
    """
    Predict on test data (not submission data from test folder) and print reports, plot results etc.
    """
     
    def __init__(self, model, training, generator, y_true):
        self.training = training
        self.generator = generator
        
        # predict the data
        self.generator.reset()
        self.y_pred_raw = model.predict_generator(self.generator, steps=STEP_SIZE_TEST)
        self.y_pred_raw = np.array(self.y_pred_raw).reshape(667, 4)
        self.pred_bool = (self.y_pred_raw>0.5)
        self.y_pred = self.pred_bool.astype(int)
        self.y_true=y_true[:len(self.y_pred)]  
        
    def Accuracy(self):
        temp = 0
        for i in range(self.y_true.shape[0]):
            temp += sum(np.logical_and(self.y_true[i], self.y_pred[i])) / sum(np.logical_or(self.y_true[i], self.y_pred[i]))
        return temp / self.y_true.shape[0]

    def Precision(self):
        temp = 0
        for i in range(self.y_true.shape[0]):
            if sum(self.y_true[i]) == 0:
                continue
            temp+= sum(np.logical_and(self.y_true[i], self.y_pred[i]))/ sum(self.y_true[i])
        return temp/ self.y_true.shape[0]

    def Recall(self):
        temp = 0
        for i in range(self.y_true.shape[0]):
            if sum(self.y_pred[i]) == 0:
                continue
            temp+= sum(np.logical_and(self.y_true[i], self.y_pred[i]))/ sum(self.y_pred[i])
        return temp/ self.y_true.shape[0]

    def F1Measure(self):
        temp = 0
        for i in range(self.y_true.shape[0]):
            if (sum(self.y_true[i]) == 0) and (sum(self.y_pred[i]) == 0):
                continue
        temp+= (2*sum(np.logical_and(self.y_true[i], self.y_pred[i])))/ (sum(self.y_true[i])+sum(self.y_pred[i]))
        return temp/ self.y_true.shape[0]

In [55]:
# Create evaluator instance
evaluator_mo = Evaluate(model1, history_multioutput, test_generator, test_ml[["defect1", "defect2", "defect3", "defect4"]].values)

print('Test Accuracy', evaluator_mo.Accuracy())
print('Test Precision', evaluator_mo.Precision())
print('Test Recall', evaluator_mo.Recall())
print('Test F1score', evaluator_mo.F1Measure())

# Pre-Trained Resnet50 and Xception Model

In [27]:
from keras.applications.resnet50 import ResNet50, preprocess_input

model2 = Sequential()
model2.add(ResNet50(include_top=False, input_shape=(128, 256, 3)))

for layer in model2.layers[:-5]:
    layer.trainable=False
    
model2.add(Dropout(0.5))
model2.add(Flatten())
model2.add(Dense(128,activation="relu"))
model2.add(Dropout(0.3))
model2.add(Dense(256,activation="relu"))
model2.add(Dense(4, activation="sigmoid"))
model2.compile(optimizer=optimizers.rmsprop(lr=0.0001, decay=1e-6), loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
from keras.applications.xception import Xception

model3 = Sequential()
model3.add(Xception(include_top=False, input_shape=(128, 256, 3)))

for layer in model2.layers[:-5]:
    layer.trainable=False
    
model3.add(Dropout(0.5))
model3.add(Flatten())
model3.add(Dense(128,activation="relu"))
model3.add(Dropout(0.3))
model3.add(Dense(256,activation="relu"))
model3.add(Dense(4,activation="sigmoid"))
model3.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])

In [31]:
earlystopper = EarlyStopping(monitor='val_acc', patience=10, verbose=1)

# Save the best model during the traning
checkpointer = ModelCheckpoint('best_model1.h5',
                               monitor='val_acc',
                               verbose=1,
                               save_best_only=True,
                               save_weights_only=True)

In [32]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

history_resnet = model2.fit_generator(generator=train_generator,
                            steps_per_epoch=STEP_SIZE_TRAIN,
                            validation_data=valid_generator,
                            validation_steps=STEP_SIZE_VALID,
                            epochs=30,
                            callbacks=[earlystopper, checkpointer])

In [33]:
# Create evaluator instance
evaluator_resnet = Evaluator(model2, history_resnet, test_generator, test_ml[["defect1", "defect2", "defect3", "defect4"]].values)

# Draw accuracy and loss charts
evaluator.plot_history()

print('Test Accuracy', evaluator_resnet.Accuracy())
print('Test Precision', evaluator_resnet.Precision())
print('Test Recall', evaluator_resnet.Recall())
print('Test F1score', evaluator_resnet.F1Measure())

In [34]:
history_X = model3.fit_generator(generator=train_generator,
                                        steps_per_epoch=STEP_SIZE_TRAIN,
                                        validation_data=valid_generator,
                                        validation_steps=STEP_SIZE_VALID,
                                        epochs=30,
                                        callbacks=[earlystopper, checkpointer])

In [35]:
# Create evaluator instance
evaluator_X = Evaluate(model3, history_X, test_generator, test_ml[["defect1", "defect2", "defect3", "defect4"]].values)

# Draw accuracy and loss charts
evaluator.plot_history()

print('Test Accuracy', evaluator_X .Accuracy())
print('Test Precision', evaluator_X.Precision())
print('Test Recall', evaluator_X.Recall())
print('Test F1score', evaluator_X.F1Measure())